# Выбор локации для скважины

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
geo_data_1 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
geo_data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
def model_regression(data):
    target = data['product']
    features = data.drop(['id','product'], axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(features, target,
                                                                                  test_size=0.25,
                                                                                  random_state=12345)
    scaler = StandardScaler()
    pd.options.mode.chained_assignment = None
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    return (features_train, features_valid, target_train, target_valid)

Вывод: данные загружены и подготовлены для последующего обучения.

## Обучение и проверка модели

### Первый регион

In [6]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = model_regression(geo_data_1)

model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)

predicted_valid_1 = model_1.predict(features_valid_1)
mse_1 = mean_squared_error(target_valid_1, predicted_valid_1)
mae_1 = mean_absolute_error(target_valid_1, predicted_valid_1)

print('Средний прогнозируемый запас сырья:', predicted_valid_1.mean())
print('RMSE = ', mse_1 ** 0.5)
print('MAE = ', mae_1)

Средний прогнозируемый запас сырья: 92.59256778438035
RMSE =  37.5794217150813
MAE =  30.919600777151313


### Второй регион

In [7]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = model_regression(geo_data_2)

model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)

predicted_valid_2 = model_2.predict(features_valid_2)
mse_2 = mean_squared_error(target_valid_2, predicted_valid_2)
mae_2 = mean_absolute_error(target_valid_2, predicted_valid_2)

print('Средний прогнозируемый запас сырья:', predicted_valid_2.mean())
print('RMSE = ', mse_2 ** 0.5)
print('MAE = ', mae_2)

Средний прогнозируемый запас сырья: 68.728546895446
RMSE =  0.893099286775617
MAE =  0.7187662442124758


### Третий регион

In [8]:
features_train_3, features_valid_3, target_train_3, target_valid_3 = model_regression(geo_data_3)

model_3 = LinearRegression()
model_3.fit(features_train_3, target_train_3)

predicted_valid_3 = model_3.predict(features_valid_3)
mse_3 = mean_squared_error(target_valid_3, predicted_valid_3)
mae_3 = mean_absolute_error(target_valid_3, predicted_valid_3)

print('Средний прогнозируемый запас сырья:', predicted_valid_3.mean())
print('RMSE = ', mse_3 ** 0.5)
print('MAE = ', mae_3)

Средний прогнозируемый запас сырья: 94.96504596800489
RMSE =  40.02970873393434
MAE =  32.792652105481814


Вывод: средний прогнозируемый запас сырья меньше всего во втором регионе, однако погрешность в нем меньше всего, в первом и третьем регионах средний прогнозируемый запас сырья выше, но гароздо больше погрешность.

## Подготовка к расчёту прибыли

In [9]:
BUDGET = 10_000_000_000
INCOME = 450_000
N = 200

Примем что себестоимость разработки новой скважины равна:

In [10]:
COST_PRICE = BUDGET / N
print(COST_PRICE)

50000000.0


Объем сырья для безубыточной разработки равен:

In [11]:
BREAKEVEN_POINT = COST_PRICE / INCOME
print(BREAKEVEN_POINT)

111.11111111111111


Вывод: средний объем сырья во всех регионах ниже объема сырья которого достаточно для безубыточной разработки, следовательно для разработки необходимо выбирать скважины с максимальными запасами которые выше точки безубыточности.

Напишем функцию расчета прибыли:

In [12]:
def profit(predicted_volume, target):
    predicted_volume = predicted_volume.sort_values(ascending=False)
    product_volume = target[predicted_volume.index]
    product_volume = product_volume[:N]
    profit = product_volume * INCOME - COST_PRICE
    return profit.sum()

## Расчёт прибыли и рисков 

### Первый регион

In [13]:
state = np.random.RandomState(12345)

values = []
for i in range(1000):
    predicted_valid_1 = pd.Series(predicted_valid_1)
    target_valid_1 = pd.Series(target_valid_1).reset_index(drop=True)
    target_subsample = target_valid_1.sample(n=500,
                                             replace=True,
                                             random_state=state)
    predicted_subsample = predicted_valid_1[target_subsample.index]
    values.append(profit(predicted_subsample, target_subsample))

In [14]:
values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()
print("Средняя прибыль:", mean)
print("2.5%-квантиль:", lower)
print("97.5%-квантиль:", upper)
loss = values[values<0].count() / len(values)
print(f'Риск убытков:', loss)

Средняя прибыль: 425938526.91059244
2.5%-квантиль: -102090094.83793713
97.5%-квантиль: 947976353.358369
Риск убытков: 0.06


### Второй регион

In [15]:
values = []
for i in range(1000):
    predicted_valid_2 = pd.Series(predicted_valid_2)
    target_valid_2 = pd.Series(target_valid_2).reset_index(drop=True)
    target_subsample = target_valid_2.sample(n=500,
                                             replace=True,
                                             random_state=state)
    predicted_subsample = predicted_valid_2[target_subsample.index]
    values.append(profit(predicted_subsample, target_subsample))

In [16]:
values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()
print("Средняя прибыль:", mean)
print("2.5%-квантиль:", lower)
print("97.5%-квантиль:", upper)
loss = values[values<0].count() / len(values)
print(f'Риск убытков:', loss)

Средняя прибыль: 518259493.6973247
2.5%-квантиль: 128123231.43308508
97.5%-квантиль: 953612982.066907
Риск убытков: 0.003


### Третий регион

In [17]:
values = []
for i in range(1000):
    predicted_valid_3 = pd.Series(predicted_valid_3)
    target_valid_3 = pd.Series(target_valid_3).reset_index(drop=True)
    target_subsample = target_valid_3.sample(n=500,
                                             replace=True,
                                             random_state=state)
    predicted_subsample = predicted_valid_3[target_subsample.index]
    values.append(profit(predicted_subsample, target_subsample))

In [18]:
values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()
print("Средняя прибыль:", mean)
print("2.5%-квантиль:", lower)
print("97.5%-квантиль:", upper)
loss = values[values<0].count() / len(values)
print(f'Риск убытков:', loss)

Средняя прибыль: 420194005.3440501
2.5%-квантиль: -115852609.1600105
97.5%-квантиль: 989629939.8445754
Риск убытков: 0.062


Вывод: Второй регион обладает наименьшим риском убытков из всех, а также средняя прибыль выше чем у других регионов, следовательно к разработке предлагается второй регион.